In [2]:
import numpy as np
import theano.tensor as T
from theano import function
import theano

from theano import config
config.compute_test_value = 'raise'

# updates = OrderedDict()

In [ ]:

class Simple_Model:
    """ Simple emission model without CNN
    word embedding layer -> ReLU layer -> softmax layer
    """
    
    def __init__(self, input_size, output_size):

        # word embedding layer
        x_input = T.vector().astype(config.floatX)
        x_input_w = T.matrix().astype(config.floatX)
        x_input_b = T.vector().astype(config.floatX)
        word_embedding_layer = T.dot(x_input_w, x_input) + x_input_b
        # ReLU layer
        relu_layer = T.nnet.relu(word_embedding_layer)
        # Softmax layer
        softmax_layer = T.nnet.softmax(relu_layer)
        
        self.model = theano.function(x_input, outputs=softmax_layer)
        
        self.x_input_w = np.random.random((x_input.shape()[0], input_size)).astype(config.floatX)
        self.x_input_b = np.random.random(input_size).astype(config.floatX)
    
    def 

In [ ]:
input_size = 512